In [8]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from matplotlib.colors import LogNorm
from PIL import Image

# Generate fake data
x = np.random.normal(size=500)
y = x * 3 + np.random.normal(size=500)

# Calculate the point density
xy = np.vstack([x, y])
z = gaussian_kde(xy)(xy)

img = Image.fromarray(xy)
img.show()

In [ ]:
# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]

fig, ax = plt.subplots()
plt.scatter(x, y, c=z, s=20, cmap='Spectral')
plt.colorbar()
plt.show()

In [ ]:
from PIL import Image

img = Image.fromarray(z)
img.show()

In [ ]:
import seaborn as sns

sns.kdeplot(x=x, y=y, fill=True, cmap='Spectral', cbar=True)

In [ ]:
# !pip install mpl-scatter-density
import mpl_scatter_density

N = 100000
x = np.random.normal(size=N)
y = x * 3 + np.random.normal(size=N)

# 绘制二维散点密度图
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='scatter_density')
density = ax.scatter_density(x, y, cmap='Spectral_r')
ax.set_xlim(-3, 3)
ax.set_ylim(-10, 10)
fig.colorbar(density, label='Number of points per pixel')
fig.savefig('gaussian.png')

In [ ]:
import matplotlib.colors as mcolors

norm = mcolors.TwoSlopeNorm(vmin=-1, vmax=60, vcenter=0)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='scatter_density')
density = ax.scatter_density(x, y, norm=norm, cmap=plt.cm.RdBu)
ax.set_xlim(-3, 3)
ax.set_ylim(-10, 10)
fig.colorbar(density, label='Number of points per pixel')
fig.savefig('gaussian_color_coded.png')

In [ ]:
array = np.where(array > 0, array, np.nan)
# 或者
array = np.ma.masked_array(array, mask=(array <= 0))

In [ ]:
from fast_histogram import histogram2d

ymin, ymax = y.min(), y.max()
xmin, xmax = x.min(), x.max()

array = histogram2d(y, x, bins=10, range=((ymin, ymax), (xmin, xmax)))

In [ ]:
from osgeo import gdal
from einops import rearrange
import os

# os.open("../../../data/out/sentinel/s2_20171029",flags=gdal.GA_ReadOnly)

s2_16 = gdal.Open('../data/s2_16_01.tif')
s2_16_arr = s2_16.ReadAsArray()
s2_17 = gdal.Open('../data/s2_17_01.tif')
s2_17_arr = s2_17.ReadAsArray()
mod = gdal.Open('../data/modis_01_up.tif')
mod_arr = mod.ReadAsArray()

s2_16_arr = rearrange(s2_16_arr, "c h w -> h w c")
s2_17_arr = rearrange(s2_17_arr, "c h w -> h w c")
mod_arr = rearrange(mod_arr, "c h w -> h w c")

In [ ]:
s2_17_arr.dtype

In [ ]:
print(s2_16.RasterXSize, s2_16.RasterYSize)
print(mod.RasterXSize, mod.RasterYSize)

In [ ]:
s2_16_arr.shape, mod_arr.shape, s2_17_arr.shape

In [ ]:
s2_17_arr.shape

In [ ]:
import numpy as np

img = np.concatenate((mod_arr, s2_16_arr, s2_17_arr), axis=-1)
img.shape

In [ ]:
np.mean(s2_16_arr, axis=(0, 1))

In [ ]:
np.mean(mod_arr, axis=(0, 1))

In [ ]:
np.mean(img, axis=(0, 1))

In [ ]:
from random import randrange


def save_random_patches(mod_img, s2_train_img, s2_label_img, save_path, patch_size=(200, 200), train_size=100):
    """
    输入img的shape为h,w,c
    :param mod_img: 
    :param s2_train_img: 
    :param s2_label_img: 
    :param save_path: 
    :param patch_size: 
    :param train_size: 
    :return: 
    """
    img = np.concatenate((mod_img, s2_train_img, s2_label_img), axis=-1)  # h,w,c

    band = img.shape[-1]

    mod = np.zeros((train_size, patch_size[0], patch_size[1], band)).astype(np.float32)
    s2 = np.zeros((train_size, patch_size[0], patch_size[1], band)).astype(np.float32)
    label = np.zeros((train_size, patch_size[0], patch_size[1], band)).astype(np.float32)

    for i in range(0, train_size):
        # while True:
        upper_left_x = randrange(0, img.shape[0] - patch_size[0])
        upper_left_y = randrange(0, img.shape[1] - patch_size[1])
        crop_point = [upper_left_x,
                      upper_left_y,
                      upper_left_x + patch_size[0],
                      upper_left_y + patch_size[1]]
        patch = img[crop_point[0]:crop_point[2], crop_point[1]:crop_point[3]]  # h,w,c
        mod[i] = patch[:, :, :, :7]
        s2[i] = patch[:, :, :, 7:11]
        label[i] = patch[:, :, :, 11:]

    np.save(save_path + '/mod', mod)
    del mod
    np.save(save_path + '/s2', s2)
    del s2
    np.save(save_path + '/label', label)
    del label

    print('Done!')

In [ ]:
save_random_patches(mod_arr, s2_16_arr, s2_17_arr, save_path="../data", patch_size=(200, 200), train_size=100)

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np


def truncated_linear_stretch(image, truncated_value, max_out=255, min_out=0):
    def gray_process(gray):
        truncated_down = np.percentile(gray, truncated_value)
        truncated_up = np.percentile(gray, 100 - truncated_value)
        gray = (gray - truncated_down) / (truncated_up - truncated_down) * (max_out - min_out) + min_out
        gray[gray < min_out] = min_out
        gray[gray > max_out] = max_out
        if (max_out <= 255):
            gray = np.uint8(gray)
        elif (max_out <= 65535):
            gray = np.uint16(gray)
        return gray

    #  如果是多波段
    if (len(image.shape) == 3):
        image_stretch = []
        for i in range(image.shape[0]):
            gray = gray_process(image[i])
            image_stretch.append(gray)
        image_stretch = np.array(image_stretch)
    #  如果是单波段
    else:
        image_stretch = gray_process(image)
    return image_stretch


def display_rgb(image_path, band_indices):
    # 打开TIFF文件
    dataset = gdal.Open(image_path, gdal.GA_ReadOnly)

    if dataset is None:
        print("无法打开文件")
        return

    # 读取选定的三个波段数据
    band1 = dataset.GetRasterBand(band_indices[0])
    band2 = dataset.GetRasterBand(band_indices[1])
    band3 = dataset.GetRasterBand(band_indices[2])

    data1 = band1.ReadAsArray()
    data2 = band2.ReadAsArray()
    data3 = band3.ReadAsArray()

    data1 = truncated_linear_stretch(data1, 2)
    data1 = truncated_linear_stretch(data1, 2)
    data1 = truncated_linear_stretch(data1, 2)

    # 关闭数据集
    dataset = None

    # 创建RGB图像
    rgb_image = np.dstack((data1, data2, data3))
    print(rgb_image.shape)
    # 显示图像
    plt.imshow(rgb_image)
    plt.title(band_indices)
    plt.show()


# 替换为你的TIFF文件路径
image_path = "../data/s2/s2_0.tif"

band_list = [(1, 2, 3), (1, 2, 4), (1, 3, 2), (1, 3, 4), (1, 4, 2), (1, 4, 3), (2, 1, 3), (2, 1, 4), (2, 3, 1), (2, 3, 4), (2, 4, 1), (2, 4, 3),
             (3, 1, 2), (3, 1, 4), (3, 2, 1), (3, 2, 4), (3, 4, 1), (3, 4, 2), (4, 1, 2), (4, 1, 3), (4, 2, 1), (4, 2, 3), (4, 3, 1), (4, 3, 2)]
for bands in band_list:
    # 选择要显示的三个波段的索引（默认为1、2、3）
    selected_band_indices = bands

    # 显示RGB图像
    display_rgb(image_path, selected_band_indices)

In [28]:
import torch
import random

random.seed(0)
factor = 4

a = torch.randn(1, 256, 256)
print(a.shape)



torch.Size([1, 256, 256])


In [29]:
a_nearest = torch.nn.functional.interpolate(a, scale_factor=1 / factor, mode='nearest')
print(a_nearest.shape)
a_nearest_up = torch.nn.functional.interpolate(a_nearest, scale_factor=factor, mode='nearest')
print(a_nearest_up.shape, torch.unique(a_nearest_up == a))

torch.Size([1, 256, 64])
torch.Size([1, 256, 256]) tensor([False,  True])


In [30]:
a_linear = torch.nn.functional.interpolate(a, scale_factor=1 / factor, mode='linear')
print(a_linear.shape)
a_linear_up = torch.nn.functional.interpolate(a_linear, scale_factor=factor, mode='linear')
print(a_linear_up.shape, torch.unique(a_linear_up == a))

torch.Size([1, 256, 64])
torch.Size([1, 256, 256]) tensor([False])


In [31]:
a = torch.randn(1, 1, 256, 256)
print(a.shape)

torch.Size([1, 1, 256, 256])


In [32]:
a_bilinear = torch.nn.functional.interpolate(a, scale_factor=1 / factor, mode='bilinear')
print(a_bilinear.shape)
a_bilinear_up = torch.nn.functional.interpolate(a_bilinear, scale_factor=factor, mode='bilinear')
print(a_bilinear_up.shape, torch.unique(a_bilinear_up == a))

torch.Size([1, 1, 64, 64])
torch.Size([1, 1, 256, 256]) tensor([False])


In [33]:
a_bicubic = torch.nn.functional.interpolate(a, scale_factor=1 / factor, mode='bicubic')
print(a_bicubic.shape)
a_bicubic_up = torch.nn.functional.interpolate(a_bicubic, scale_factor=factor, mode='bicubic')
print(a_bicubic_up.shape, torch.unique(a_bicubic_up == a))

torch.Size([1, 1, 64, 64])
torch.Size([1, 1, 256, 256]) tensor([False])
